# User Uploaded Tables

**Description:** Use the TAP upload functionality for user-supplied tables and join them with DP.02 catalogs.

### 1.1. Import packages

In [ ]:
import os
import getpass
import matplotlib.pyplot as plt
import numpy as np
import pyvo
from astropy.table import Table
import pandas as pd
from lsst.rsp import get_tap_service

### 1.2. Define parameters

Set a few style parameters for the plots.

In [ ]:
plt.style.use('tableau-colorblind10') 
params = {'axes.labelsize': 12,
          'font.size': 12,
          'legend.fontsize': 10}
plt.rcParams.update(params)

In [ ]:
plot_filter_colors_white_background = {
    'u': '#0c71ff',
    'g': '#49be61',
    'r': '#c61c00',
    'i': '#ffc200',
    'z': '#f341a2',
    'y': '#5d0000'
}

plot_symbols = {
    'u': 'o',
    'g': '^',
    'r': 'v',
    'i': 's',
    'z': '*',
    'y': 'p'
}

Define the path to to input files, which are stored in a shared directory in the Notebook Aspect.

Start the TAP service and assert that it exists.

In [ ]:
rsp_tap = get_tap_service("tap")
assert rsp_tap is not None

### 1.2. Create upload data (as a string)

In [ ]:
query = """
        SELECT TOP 10 * 
        FROM dp02_dc2_catalogs.DiaSource
        """

In [ ]:
job = rsp_tap.submit_job(query)
job.run()

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()

In [ ]:
print(results)

In [ ]:
results = job.fetch_result().to_table()

# coords
coords_string = "diaSourceId\tcoord_ra\tcoord_dec\tmidPointTai\n" + "\n".join(
    f"{row['diaSourceId']}\t{row['coord_ra']}\t{row['coord_dec']}\t{row['midPointTai']}" 
    for _, row in enumerate(results)
)

## 2. Demonstrate simple query that searches the uploaded table

In [ ]:
ut1 = Table.read(coords_string, format='ascii.basic')

In [ ]:
query = """
        SELECT * FROM TAP_UPLOAD.ut1 AS ut1
        WHERE ut1.coord_ra > 10
        """

In [ ]:
job = rsp_tap.submit_job(query, uploads={"ut1": ut1})
job.run()

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
results

## 3. Spatial and temporal cross-match to diaSources

In [ ]:
ut1 = Table.read(coords_string, format='ascii.basic')

In [ ]:
query = """
        SELECT dias.coord_ra, dias.coord_dec, dias.midPointTai, dias.diaSourceId,
        ut1.coord_ra AS ut1_ra, ut1.coord_dec AS ut1_dec, ut1.midPointTai AS ut1_mjd, ut1.diaSourceId AS ut1_id
        FROM dp02_dc2_catalogs.DiaSource AS dias, TAP_UPLOAD.ut1 AS ut1
        WHERE CONTAINS(POINT('ICRS', dias.coord_ra, dias.coord_dec),
        CIRCLE('ICRS', ut1.coord_ra, ut1.coord_dec, 0.00278))=1
        AND ABS(dias.midPointTai - ut1.midPointTai) < 0.5
        ORDER BY dias.diaSourceId
        """

Create the job by submitting the query and then run it asynchronously.

In [ ]:
job = rsp_tap.submit_job(query, uploads={"ut1": ut1})
job.run()

Check that the job is completed.

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Retrieve the results and display them.

In [ ]:
results = job.fetch_result().to_table()
results

## 3. Object identifier cross-match to diaSources

This section demonstrates how to upload a user-supplied table and join it with a DP1 table.

In [ ]:
ut2 = Table.read(coords_string, format='ascii.basic')

In [ ]:
query = """
    SELECT *
    FROM dp02_dc2_catalogs.DiaSource AS dias, TAP_UPLOAD.ut2 AS ut2
    WHERE ut2.diaSourceId = dias.diaSourceId   
    """

In [ ]:
job = rsp_tap.submit_job(query, uploads={"ut2": ut2})
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
results

Print the unique `ssObjectId` and how many detections each had.

In [ ]:
uniqueIds, counts = np.unique(results['diaSourceId'], return_counts=True)
for uniqueId, count in zip(uniqueIds, counts):
    print("diaSourceId:", uniqueId, "Number of detections:", count)

### 3.1. Plot the sky distribution of LSST detections

Create a plot showing sky distribution of the detections of these two unique objects over 10 years.

In [ ]:
fig = plt.figure(figsize=(6, 4))
for uniqueId in uniqueIds:
    tx = results['diaSourceId'] == uniqueId
    plt.plot(results['coord_ra'][tx], results['coord_dec'][tx],
             'o', ms=5, alpha=0.3, mew=0, label=str(uniqueId))
plt.xlabel('Right Ascension [deg]')
plt.ylabel('Declination [deg]')
plt.title('LSST detections of two moving objects')
plt.legend(loc='lower left')
plt.show()

> Figure 2: Above, the coordinates of the LSST difference-image detections from the `diaSource` catalog for the two moving objects listed in the user-uploaded table.

